# MWE for eqtl
This serve as a demostration and records of how to generate and what to use the xqtl calling and discovery pipeline. The samples that we use are the first 50 samples in the [ftp server](http://ftp.ebi.ac.uk/pub/databases/arrayexpress/data/experiment/GEUV/E-GEUV-1/processed/) of [This project](https://www.ebi.ac.uk/arrayexpress/experiments/E-GEUV-1/). Each of the bam file was downloaded using wget. The bam file will be subset to include only chr 21 and 22,  convert to fastq file, and then starts from running our RNA-calling pipeline.